<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Visión por Computadora</center></span>
## <span style="color:red"><center>Redes Residuales</center></span>

##   <span style="color:blue">Profesores</span>

### Coordinador


- Campo Elías Pardo Turriago, cepardot@unal.edu.co 

### Conferencistas


- Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
- Daniel  Montenegro, Msc, dextronomo@gmail.com 
- Oleg Jarma, Estadístico, ojarmam@unal.edu.co 

## <span style="color:blue">Asesora Medios y Marketing digital</span>

- Maria del Pilar Montenegro, pmontenegro88@gmail.com 



## <span style="color:blue">Asistentes</span>



- Nayibe Yesenia Arias, naariasc@unal.edu.co
- Venus Celeste Puertas, vpuertasg@unal.edu.co 

## <span style="color:blue">Introducción</span>

Hablemos un poco de mitología

<figure>
<center>
<img src="../Imagenes/stairway_to_heaven.png" width="500" height="600" align="center"/> 
</center>
</figure>

En un famoso mito Lepcha, se destruye una escalera al cielo después de un efecto de teléfono roto. Esta conclusión puede transferirse de cierta forma a la construcción de modelos. 

Desde el año 2012, con la creación de la arquitectura AlexNet, ganadora del "ImageNet Large Scale Visual Recognition Challenge", varios ingenieros de Aprendizaje automatizado llegaron a una conclusión: Más capas = mayor precisión. Según lo que hemos estudiado, esto tiene sentido, pero en la realidad esto es mucho más complicado.

Vamos a ver un método para permitir aplicar más capas en los modelos de visión, y una arquitectura completa que permite trabajar con grandes cantidades de capas y procesamiento.




## <span style="color:blue">Normalización por Lotes</span>

Durante el proceso de desarrollo y entrenamiento de modelos de redes neuronales, nos encontramos con unos cuantos problemas.

- Sesgo que pueden generar los datos debido a diferentes escalas.
- las magnitudes de las capas de una red pueden variar de una a otra, causando problemas de convergencia
- Entre más compleja sea una red, más sencillo es que exista sobreajuste.

Ya hemos visto como "mitigar" esto utilizando funciones como `StandardScaler` o `MixMaxScaler` y capas `dropout`, pero estos tienen sus límites. Aunque podemos tomar cierta inspiración sobre cómo funcionan estos para encontrar una mejor solución.

¿Qué tal si aplicamos la idea de "escalimiento" de manera individual a las capas que queramos?

<figure>
<center>
<img src="../Imagenes/batch_normalization_1.png" width="700" height="400" align="center"/> 
</center>
</figure>

En el caso tradicional solo se hace la "normalización" antes de que los datos pasen por la capa de entrada. Pero dentro de la red, todos las capas reciben datos de entrada. La idea es normalizar estos inputs antes de que sean procesados en la siguiente capa.

¿Cómo hacemos esto? se obtienen las estadísticas del lote actual, y a cada dato se le resta el promedio y se divide sobre la desviación estándar. De esta manera los datos seguirán una distribución Normal estándar. Finalemente se introducen dos parámetros que van a ser aprendidos por el modelo: El escalamiento $\gamma$, y posicionamiento $\beta$. Esto con la intención de poder "modular" la distribución de los datos, encontrando aquella que mejore el proceso de convergencia.

Seamos más exactos con lo que sucede

sobre cada entrada $\mathbf{x}$ que pertenece al lote $\mathcal{B}$, la Normalización por lotes (BN) transforma a $\mathbf{x}$ de la siguiente forma:

\begin{align}
    \mathrm{BN}(\mathbf{x}) = \boldsymbol{\gamma} \odot \frac{\mathbf{x} - \hat{\boldsymbol{\mu}}_\mathcal{B}}{\hat{\boldsymbol{\sigma}}_\mathcal{B}} + \boldsymbol{\beta}.
\end{align}

donde: 

\begin{split}
    \begin{aligned} 
        \hat{\boldsymbol{\mu}}_\mathcal{B} &= \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} \mathbf{x},\\
\hat{\boldsymbol{\sigma}}_\mathcal{B}^2 &= \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} (\mathbf{x} - \hat{\boldsymbol{\mu}}_{\mathcal{B}})^2 + \epsilon.      
    \end{aligned}
\end{split}

Vamos a hacer un prqueño ejemplo comparativo. Aplicando Esta normalización por lotes a nuestro modelo convolucional más clásico: LeNet

In [11]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
train_set = torchvision.datasets.FashionMNIST("/mnt/storage/Datasets", download=True, transform=
                                                transforms.Compose([transforms.ToTensor()]))
test_set = torchvision.datasets.FashionMNIST("/mnt/storage/Datasets", download=True, train=False, transform=
                                               transforms.Compose([transforms.ToTensor()])) 

In [28]:
train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=100)
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=100)

batch = next(iter(test_loader))
images, labels = batch
print(type(images), type(labels))
print(images.shape, labels.shape)

<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([100, 1, 28, 28]) torch.Size([100])


In [29]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=2), 
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
    
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
    
        self.fc1 = nn.Linear(16*5*5, 120)
        self.sigmoid1 = nn.Sigmoid()
        self.fc2 = nn.Linear(120, 84)
        self.sigmoid2 = nn.Sigmoid()
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.sigmoid1(out)
        out = self.fc2(out)
        out = self.sigmoid2(out)
        out = self.fc3(out)

        return out
    

In [30]:
model = LeNet()
model.to(device)

error = nn.CrossEntropyLoss()

learning_rate = 1.0
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(model)

LeNet(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (sigmoid1): Sigmoid()
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (sigmoid2): Sigmoid()
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [31]:
num_epochs = 10
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
    
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)
        
        outputs = model(train)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
    
        count += 1
    
    
        if not (count % 50):
            total = 0
            correct = 0
        
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
            
                test = Variable(images.view(100, 1, 28, 28))
            
                outputs = model(test)
            
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()
            
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))

Iteration: 500, Loss: 2.307828187942505, Accuracy: 10.0%
Iteration: 1000, Loss: 2.2942955493927, Accuracy: 10.0%
Iteration: 1500, Loss: 2.301664113998413, Accuracy: 10.0%
Iteration: 2000, Loss: 1.1822646856307983, Accuracy: 56.13999938964844%
Iteration: 2500, Loss: 0.8801200985908508, Accuracy: 62.98999786376953%
Iteration: 3000, Loss: 0.6563175320625305, Accuracy: 70.77999877929688%
Iteration: 3500, Loss: 0.624728798866272, Accuracy: 73.91999816894531%
Iteration: 4000, Loss: 0.5167486071586609, Accuracy: 79.12999725341797%
Iteration: 4500, Loss: 0.4693790376186371, Accuracy: 81.5199966430664%
Iteration: 5000, Loss: 0.5594117641448975, Accuracy: 78.91999816894531%
Iteration: 5500, Loss: 0.38821494579315186, Accuracy: 82.88999938964844%
Iteration: 6000, Loss: 0.40123307704925537, Accuracy: 84.27999877929688%


Agreguemos ahora unas capas de normalización de lotes

In [37]:
class LeNet_Norm(nn.Module):
    def __init__(self):
        super(LeNet_Norm, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=2), 
            nn.Sigmoid(),
            nn.BatchNorm2d(6),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
    
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.BatchNorm2d(16),
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )
    
        self.fc1 = nn.Linear(16*5*5, 120)
        self.batchnorm1 = nn.BatchNorm1d(120)
        self.sigmoid1 = nn.Sigmoid()
        self.fc2 = nn.Linear(120, 84)
        self.batchnorm2 = nn.BatchNorm1d(84)
        self.sigmoid2 = nn.Sigmoid()
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.batchnorm1(out)
        out = self.sigmoid1(out)
        out = self.fc2(out)
        out = self.batchnorm2(out)
        out = self.sigmoid2(out)
        out = self.fc3(out)

        return out

In [38]:
model = LeNet_Norm()
model.to(device)

error = nn.CrossEntropyLoss()

learning_rate = 1.0
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
print(model)

LeNet_Norm(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Sigmoid()
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (batchnorm1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sigmoid1): Sigmoid()
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (batchnorm2): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (sigmoid2): Sigmoid()
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [39]:
num_epochs = 10
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
    
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)
        
        outputs = model(train)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
    
        count += 1

    
        if not (count % 50):   
            total = 0
            correct = 0
        
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
            
                test = Variable(images.view(100, 1, 28, 28))
            
                outputs = model(test)
            
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()
            
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))

Iteration: 500, Loss: 0.5263475179672241, Accuracy: 82.06999969482422%
Iteration: 1000, Loss: 0.3830808997154236, Accuracy: 85.33999633789062%
Iteration: 1500, Loss: 0.33494433760643005, Accuracy: 87.07999420166016%
Iteration: 2000, Loss: 0.39517685770988464, Accuracy: 85.7699966430664%
Iteration: 2500, Loss: 0.23648852109909058, Accuracy: 86.8699951171875%
Iteration: 3000, Loss: 0.22149448096752167, Accuracy: 89.00999450683594%
Iteration: 3500, Loss: 0.3859484791755676, Accuracy: 88.83999633789062%
Iteration: 4000, Loss: 0.24774448573589325, Accuracy: 89.50999450683594%
Iteration: 4500, Loss: 0.22523845732212067, Accuracy: 90.00999450683594%
Iteration: 5000, Loss: 0.31093594431877136, Accuracy: 88.48999786376953%
Iteration: 5500, Loss: 0.1495741903781891, Accuracy: 89.07999420166016%
Iteration: 6000, Loss: 0.19472689926624298, Accuracy: 89.97999572753906%


El aumento de velocidad en la convergencia, junto con los resultados finales, nos muestra una mejora inmediata.

### <span style="color:blue">Cosas a tener en cuenta</span>

- Nadie está 100% seguro por qué esto funciona
- Es preferible no mezclar droput y esta técnica
- No es necesario poner una capa de normalización después de cada capa regular
- Están las tendencias de aplicar la normalización antes de la activación y la de aplicarla después

## <span style="color:blue">Redes Residuales</span>

Pensemos un poco sobre cómo diseñar un modelo de red neuronal: ¿Será un MLP? ¿Convolución? ¿Recurrente?. Aquí decidimos unas cuantas cosas, pero lo más relevante aquí es el número de capas. Usualmente empezamos con un número bajo, pero cuando no tenemos buenos resultados, uno de nuestros primeros pensamientos será "Agregar más capas". Y hasta cierto punto, tenemos razón.



Volviendo a lo que charlabamos en la [Introducción](#Introducción), desde el primer ganador del ImageNet Challenge en 2012, lo que se ha hecho es construir sobre estos modelos ganadores, mejorando parámetros y, agregando más capas

<figure>
<center>
<img src="../Imagenes/vgg-vs-alexnet.png" width="400" height="500" align="center"/> 
</center>
</figure>
 

### <span style="color:blue">El problema del gradiente desvaneciente</span>



En la universidad Nacional de Colombia se tiene, como indicador de rendimiento o desempeño, Promedio Aritmético Ponderado Acumulado(PAPA). Este, como dice el nombre, es un promedio de las materias cursadas, con pesos con respecto a la cantidad de créditos que vale cada una. En los inicios de la carrera universitaria, este valor es altamente volátil dependiendo de los resultados que se obtenga en los semestres, pero a medida que se avanza en la carrera, los valores altos o bajos ya no tienen tanto efecto en este.

El problema del gradiente desvaneciente puede verse de una forma similar. Entre más capas se introduzcan, más funciones de activación se aplicarán sobre los datos. Y dependiendo de qué funciones de activación se utilicen, tendrémos diferentes resultados sobre sus derivadas, y por tanto, de los pesos que aprende. A medida que se sigan aplicando más y más de estas funciones, la cantidad de lo que aprende la red se irá disminuyendo lentamente hasta que tal vez no aprenda en lo absoluto, haciendo imposible que el modelo pueda converger.

<figure>
<center>
<img src="../Imagenes/vanishing_gradient.png" width="1200" height="300" align="center"/> 
</center>
</figure>

También podemos encontrarnos con el problema opuesto, en el que el gradiente crece y crece de tal forma que los cambios en los pesos serían demasiado fuertes, causando divergencia.

Ya hay varios métodos que reducen los chances de que aparezcan estos problemas. El primero, por supuesto es el uso de la función ReLU y sus derivados, ya que la diferencia de este con su derivada es mínima. De igual forma, usar Normalización de lotes se ha mostrado útil para esto mismo.

Pero todo tiene sus límites, y siempre se busca hacer redes más robustas y más profundas. 

### <span style="color:blue">Bloques residuales</span>

<figure>
<img src="../Imagenes/residual_block.png" width="500" height="300" align="left"/> 
</figure>

<figure>
<img src="../Imagenes/residual-block.svg" width="600" height="400" align="right"/> 
</figure>

En su [respectivo paper](https://arxiv.org/abs/1512.03385), He et al. se cuestionan los problemas que vimos más adelante y la pared que causa esto para hacer redes más poderosas. su método para arreglarlo fue plantear una nueva forma en las que las redes aprenden.

En el caso tradicional, las redes neuronales aprenden, a partir de los datos de entrada $\mathbf{x}$, una función $f(\mathbf{x})$. Esto, sin ningún contexto, es esencialmente ensayo y error(o como lo llamamos nosotros, "entrenamiento"). 

La propuesta es darle a la red una pequeña "pista" o "ayuda" para aprender lo que queremos. digamos que ahora no queremos que aprenda $f(\mathbf{x})$, sino una nueva función $H(\mathbf{x})=f(\mathbf{x})-\mathbf{x}$, o mejor dicho le decimos que aprenda los "residuos" de la función. Esto es más sencillo ya que ahora está buscando una función con referencia a los datos de entrada, en lugar de buscar "humo".
Al final de este proceso, se suma el resultado con los datos de entrada "en limpio" para obtener $H(\mathbf{x})+\mathbf{x}=f(\mathbf{x})$. Esta suma final luego entra a la función de activación.

Esta arquitectura ganó el ImageNet Challenge de 2015 y el COCO Challenge.

Vamos a ver de forma rápida la versión implementada en pytorch y trataremos de hacerlo por cuenta propia

Veamos cómo está estructurado este modelo

In [41]:
torchvision.models.resnet34()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Vamos a hacer datos artificiales como forma de entender el funcionamiento completo

In [42]:
inp = torch.randn([2,3,224,224])

inp.shape

torch.Size([2, 3, 224, 224])

Primero hacemos un bloque convolucional clásico de entrada

In [44]:
conv_block = nn.Sequential(
    nn.Conv2d(3,64,kernel_size=7, stride=2, padding=3, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

out=conv_block(inp)
out.shape

torch.Size([2, 64, 56, 56])

In [47]:
list(torchvision.models.resnet34().children())[:4]

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)]

Seguimos con los bloques residuales. Aquí aplicamos dos capas convolucionales y luego sumamos el input al resultado

In [48]:
class BasicBlock(nn.Module):
    expansion = 1 #la rata de canales

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1,
                     padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample #convolución que hay que aplicar si el tamaño del input y el output difiere
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [49]:
BasicBlock(64,128)

BasicBlock(
  (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

Sigamos aplicando estos bloques a nuestros datos de ejemplo

In [54]:
BasicBlock(64,64)(conv_block(inp)).shape

torch.Size([2, 64, 56, 56])

Aquí es donde las cosas se ponen complicadas. 
A pesar de que los modelos ResNet publiciten tener hasta 100 capas en sus estructuras, estos están ordenados en "Bloques". Cada bloque tiene cierto número de capas u "operaciones" Convolution->BatchNorm->ReLU. Usualmente tiene uno o dos triplas de esta operación.
En lugar de hacer varias capas de manera explícita, hacen 4 Capas "principales", y dentro de estas es donde agregan nuevos bloques. Supuestamente logrando mantener la misma profundidad siempre.

Necesitamos entonces hacer una función que cree esas capas principales y las llene con la cantidad de bloques que le pidamos

In [55]:
def _make_layer(block, inplanes,planes, blocks, stride=1):
    downsample = None  
    if stride != 1 or inplanes != planes:
        downsample = nn.Sequential(            
            nn.Conv2d(inplanes, planes, 1, stride, bias=False),
            nn.BatchNorm2d(planes),
        )
    layers = []
    layers.append(block(inplanes, planes, stride, downsample))
    inplanes = planes
    for _ in range(1, blocks):
        layers.append(block(inplanes, planes))
    return nn.Sequential(*layers)

Hagamos una prueba de cómo definir estas capas

In [56]:
layers=[3, 4, 6, 3]

layer1 =_make_layer(BasicBlock, inplanes=64,planes=64, blocks=layers[0])
layer1

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (2): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, mome

In [57]:
layer2 = _make_layer(BasicBlock, 64, 128, layers[1], stride=2)
layer2

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-

El último tipo de bloque es el bloque de clasificación. Este tiene una capa de AvgPooling, una capa para aplicar el flatten y por último la capa clasificadora

In [58]:
num_classes=1000
nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
              nn.Linear(512 , num_classes))

Sequential(
  (0): AdaptiveAvgPool2d(output_size=(1, 1))
  (1): Linear(in_features=512, out_features=1000, bias=True)
)

con las partes completas y definidas podemos dar la clase entera

In [59]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        super().__init__()
        
        self.inplanes = 64

        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 , num_classes)


    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None  
   
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        
        self.inplanes = planes
        
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)           # 224x224
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)         # 112x112

        x = self.layer1(x)          # 56x56
        x = self.layer2(x)          # 28x28
        x = self.layer3(x)          # 14x14
        x = self.layer4(x)          # 7x7

        x = self.avgpool(x)         # 1x1
        x = torch.flatten(x, 1)     # remove 1 X 1 grid and make vector of tensor shape 
        x = self.fc(x)

        return x

In [60]:
def resnet34():
    layers=[3, 4, 6, 3]
    model = ResNet(BasicBlock, layers)
    return model

In [61]:
model=resnet34()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  